### создание графов
- чтобы создать совсем-совсем уникальные имена, нужно сделать несколько итераций с *change_non_unique_names()*
- если занимает слишком много места, сожми с _rar/joblib_

### создание embeddings
- empty

In [1]:
import joblib
import os
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.stats import norm
import time
import ast
from random import shuffle

In [2]:
# for python 2.7
import joblib
import os
import networkx as nx
import time
import ast
from random import shuffle

In [9]:
def count_non_unique_names(dic, non_uniques={}):
    for key in dic:
        if key in non_uniques:
                non_uniques[key] += 1
        else:
            non_uniques[key] = 0
        if type(dic[key]) is dict:
            count_non_unique_names(dic[key], non_uniques)
    res = {}
    for key in non_uniques:
        if non_uniques[key] > 0:
            res[key] = non_uniques[key]
    return res

def change_non_unique_names(dic, non_uniques, new_dic={}):
    for key in dic:
        if key in non_uniques:
                split = key.split('.')
                if len(split) > 1:
                    before_dot = ''.join(split[:len(split)-1])
                    new_key = before_dot + str(non_uniques[key]) + '.' + split[-1]
                else:
                    whole_name = split[0]
                    new_key = whole_name + str(non_uniques[key])
                non_uniques[key] -= 1
        else:
            new_key = key
        
        if type(dic[key]) is dict:
            temp = change_non_unique_names(dic[key], non_uniques, {})
            new_dic[new_key] = temp
        else:
            new_dic[new_key] = dic[key]
    return new_dic  
                        
def create_nodes_and_edges(G, dic, root):
    for key in dic:
        if type(dic[key]) is dict:
            G.add_node(key, label=key)
            G.add_edge(root, key)
            G = create_nodes_and_edges(G, dic[key], key)
        else:
            G.add_node(key, label=dic[key])
            G.add_edge(root, key)
    return G

In [21]:
# 
## for python 2.7
#


# chunks_path = 'C:/Users/pogorelov_g/Desktop/dataset_arsenii/data/'
chunks_path = '/Users/grigoriipogorelov/Desktop/KL/data/'
path_to_save = '/Users/grigoriipogorelov/Desktop/KL/gexf_graphs/'
# path_to_save = 'C:/Users/pogorelov_g/Desktop/work/gexf_graphs/'


files = []
files_count = 0
portion = 0.1
for filename in os.listdir(chunks_path):
    files.append(filename)
    files_count += 1
shuffle(files)
# num_chunks_to_read = int(float(files_count)*0.1)
# files = files[:num_chunks_to_read]

num_chunks_to_print = int(float(files_count)*0.1)
failed_jbls = {}
count_jbl = 0
start = time.time()
for file_ in files:
    try:
        chunk = joblib.load(chunks_path + file_)
        for i, item in enumerate(chunk):
            non_unique_nodes = count_non_unique_names(item[1], {})
            if any(non_unique_nodes):
                graph_dict = change_non_unique_names(item[1], non_unique_nodes, {})
            else:
                graph_dict = item[1]
            G = nx.Graph()
            G.add_node('APK_ROOT', label='APK_ROOT')
            G = create_nodes_and_edges(G, graph_dict, 'APK_ROOT')
            nx.write_gexf(G, path_to_save + item[0] + '.gexf')
        count_jbl += 1
        if count_jbl % num_chunks_to_print == 0:
            print('[%d] chunks are read' % count_jbl)
    except (UnicodeDecodeError, ValueError, EOFError) as e:
        print('%s jbl failed, error - %s' % (file_, e))
        failed_jbls[file_] = e
        print('in total %d failed jbls' % len(failed_jbls))

end = time.time()
    
print('downloading and creating %d gexfs is %0.2f min' % (len(files), (end - start)/60))

[2] chunks are read
downloading and creating 2 gexfs is 4.12 min


In [1]:
# get dictionary out of embeddings.txt file
path_to_embeddings = '/Users/grigoriipogorelov/Desktop/KL/subgraph2vec_graph2vec/graph2vec_tf/embeddings/gexf_graphs_dims_3_epochs_100_lr_0.3_embeddings.txt'
embeddings_str = open(path_to_embeddings, 'r').read()
embeddings_dict = ast.literal_eval(embeddings_str)
type(whip)

dict

### Отрисовка графов

In [ ]:
nx.draw(G)

[link for cell below](https://networkx.github.io/documentation/networkx-1.10/examples/drawing/index.html)

In [38]:
s = 'axmlxml'
ss = s.split('.')
if len(ss)>1:
    before_dot = ''.join(ss[:len(ss)-1])
else:
    before_dot = ss[0]
    
before_dot

'axmlxml'

In [ ]:
G=nx.balanced_tree(3,5)
pos=graphviz_layout(G,prog='twopi',args='')
plt.figure(figsize=(8,8))
nx.draw(G,pos,node_size=20,alpha=0.5,node_color="blue", with_labels=False)
plt.axis('equal')
plt.savefig('circular_tree.png')
plt.show()

### Графики

In [19]:
# chunks_path = '/Users/grigoriipogorelov/Desktop/KL/data/'
# chunk = joblib.load(chunks_path + 'chunk_5.jbl')

In [1]:
# for ch in chunk:
#     if ch[0] == 'aa945772b101c01c5586b37db5f5915b':
#         print(ch[1])
#         non_unique_nodes = count_non_unique_names(ch[1], {})
#         print()
#         print()
# #         for k, v in non_unique_nodes.items():
# #             print(k+"\t", v)
#         if any(non_unique_nodes):
#             graph_dict = change_non_unique_names(ch[1], non_unique_nodes, {})
#         else:
#             graph_dict = ch[1]
#         print(graph_dict)
#         non_unique_nodes_new = count_non_unique_names(graph_dict, {})
#         print()
#         print()
#         if any(non_unique_nodes_new):
#             for k, v in non_unique_nodes_new.items():
#                 print(k+"\t", v)
#         else:
#             print('все чикс энд пукс')

In [2]:
def count_and_unique(dict_, unique):
    count = 0
    uniques = unique
    for key in dict_:
        if key not in uniques:
            uniques[key] = 0
        else:
            uniques[key] += 1  # TODO зачем вообще суммировать? можно и без этого обойтись
        if type(dict_[key]) is dict:
            c, u = count_and_unique(dict_[key], uniques)
            count += 1 + c
            uniques = u
        else:
            count += 1
    return count, uniques


sum_errs = 0
errs = []
av_num_of_elem = 0
num_elems = []
N_points = len(chunk)
n_bins = 150

for i, ch in enumerate(chunk):
    non_unique_nodes = count_non_unique_names(ch[1], {})
    if any(non_unique_nodes):
        graph_dict = change_non_unique_names(ch[1], non_unique_nodes, {})
    else:
        graph_dict = ch[1]
    
    c, u = count_and_unique(graph_dict, {})
    sum_errs += c - len(u.keys())
    errs.append(c - len(u.keys()))
    if c != len(u.keys()):
        print(c - len(u.keys()), ch[0])
    av_num_of_elem += c
    num_elems.append(c)
    

# fig, ax = plt.subplots()
# ax.set_xlabel('errs')
# ax.set_ylabel('num_elems')
# hist = ax.hist2d(errs, num_elems, range = [[0, 1500], [0, 3000]], bins=n_bins)


# num_elems_sort = sorted(num_elems)
# errs_sort = sorted(errs)

# print('(%d, %d) - max and min num of files among all graphs' % (max(num_elems),min(num_elems)))
# print('(%d, %d) - max and min num of NOT unique file names' % (max(errs), min(errs)))
# print('{0:.0f} - average num of files'.format(av_num_of_elem/1000))
# print('{0:.0f} - average num of NOT unique file names'.format(sum_errs/1000))
# print('{} - median num of files'.format(num_elems_sort[int(len(num_elems_sort)/2)]))
# print('{} - median num of NOT unique file names'.format(errs_sort[int(len(errs_sort)/2)]))





fig, ax = plt.subplots()
ax.set_title('num_elems')
N, bins, patches  = ax.hist(num_elems, bins=n_bins)
fracs = N / N.max()
norm = colors.Normalize(fracs.min(), fracs.max())
for thisfrac, thispatch in zip(fracs, patches):
    color = plt.cm.viridis(norm(thisfrac))
    thispatch.set_facecolor(color)
    
fig, ax = plt.subplots()
ax.set_title('errs')
N, bins, patches  = ax.hist(errs, bins=n_bins)
fracs = N / N.max()
norm = colors.Normalize(fracs.min(), fracs.max())
for thisfrac, thispatch in zip(fracs, patches):
    color = plt.cm.viridis(norm(thisfrac))
    thispatch.set_facecolor(color)

In [44]:
# app = {'AndroidManifest.xml': 'AXML_PLUS',
#          'META-INF': {'CERT.RSA': 'CERT_NORM',
#           'CERT.SF': 'SIG_PLUS',
#           'MANIFEST.MF': 'MANIF_PLUS'},
#          'assets': {'AZU.R.E.2d.png': {'PNG_NORM':'xui','AZU.R.E.2d.png': 'xui'},
#           'BLUE2d.png': {'c':'cc', 'a':'aa', 'GREEN2d.png':{'mm':'cc', 'META-INF':'fff'}},
#           'CYAN2d.png': 'PNG_NORM',
#           'GREEN2d.png': 'PNG_NORM'}}

In [112]:
# dir(nx)

# g = nx.read_gexf('/Users/grigoriipogorelov/Desktop/KL/subgraph2vec:graph2vec/graph2vec_tf/data/kdd_datasets/mutag/0.gexf')
# g_to_d = nx.to_dict_of_dicts(g)
# g_to_d

# g = nx.read_gexf('/Users/grigoriipogorelov/Downloads/celegans.gexf')
# g_to_d = nx.to_dict_of_dicts(g)
# # g_to_d = nx.to_dict_of_lists(g)
# # g_to_d = nx.to_pandas_adjacency(g)
# g_to_d

# toy_d = {'a':{'c':'cc', 'a':'aa', 'm':'m'},'b':'bb', 'm':{'b':'bb', 'p':'pp'}}

# app = {'AndroidManifest.xml': 'AXML_PLUS',
#          'META-INF': {'CERT.RSA': 'CERT_NORM',
#           'CERT.SF': 'SIG_PLUS',
#           'MANIFEST.MF': 'MANIF_PLUS'},
#          'assets': {'AZURE2d.png': 'PNG_NORM',
#           'BLUE2d.png': {'c':'cc', 'a':'aa', 'm':'mm'},
#           'CYAN2d.png': 'PNG_NORM',
#           'GREEN2d.png': 'PNG_NORM'}}